In [32]:
%%writefile utils/Utils.py
from tensorflow.python.client import device_lib


def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

def get_ip(domain):
    import requests
    from bs4 import BeautifulSoup
    url = "http://ip.chinaz.com/" + domain
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    x = soup.find(class_="IcpMain02")


    
    x = x.find_all("span", class_="Whwtdhalf")
    try:
        print("%s %s" % (x[5].text, x[4].text))
    except:
        pass


get_available_gpus()
device_lib.list_local_devices()

Overwriting utils/Utils.py


In [33]:
% run -i utils/Utils.py

In [1]:
%%writefile spark_02.py
# -*- coding: utf-8 -*-
import os
import sys

import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

PROJECT_DIR = os.path.abspath('..')
SPARK_HOME = '/opt/modules'  + '/spark-2.3.0'

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME + "/python")
sys.path.append(SPARK_HOME + "/python/lib/py4j-0.10.4-src.zip")
print(sys.path)
from pyspark import SparkContext, SparkConf

appName = "spark2django"  # 应用程序名称
master = "spark://192.168.100.34:7077"  # 222.28.78.90:7077为主节点ip和端口，请换成自己的主节点主机名称
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)


def getTest():
    data = [1, 2, 3, 4, 5]
    distData = sc.parallelize(data)
    res = distData.reduce(lambda a, b: a + b)
    print (res)
    return res
def textfile():
    text_file = sc.textFile("hdfs:///sparkdir/dfile1")
    print(text_file.collect())
    df = text_file.map(lambda r: Row(r)).toDF(["line"])
    print(df)
    
def save_2_mysql():


def map_extract(element):
    file_path, content = element
    year = file_path[-8:-4]
    return [(year, i) for i in content.split("\r\n") if i]
spark = SparkSession\
    .builder\
    .appName("PythonTest")\
    .getOrCreate()

res = spark.sparkContext.wholeTextFiles('hdfs://localhost:8020/datatnt/names',
                        minPartitions=40)  \
        .map(map_extract) \
        .flatMap(lambda x: x) \
        .map(lambda x: (x[0], int(x[1].split(',')[2]))) \
        .reduceByKey(lambda x,y:x+y)

df = res.toDF(["key","num"])  #把已有数据列改成和目标mysql表的列的名字相同
# print(dir(df))
df.printSchema()
print(df.show())
df.printSchema()

df.write.format("jdbc").options(
    url="jdbc:mysql://localhost:3306/leaf",
    driver="com.mysql.cj.jdbc.Driver",
    dbtable="spark",
    user="root",
    password="xx@123456").mode('append').save()
if __name__ == '__main__':
    getTest()
    #textfile()


UsageError: %%writefile is a cell magic, but the cell body is empty.


In [2]:
# !git st